In [1179]:
import csv
import sys
import numpy as np

# train = sys.argv[1]
# test = sys.argv[2]
# train_out = sys.argv[3]
# test_out = sys.argv[4]
# metrics = sys.argv[5]
# epoch_num = sys.argv[6]
# hiddenUnits = sys.argv[7]
# model_num = sys.argv[8]
# lr = sys.argv[9]

In [1180]:
def model(a,units,lr):
    if(a == 1):
        alpha = np.random.uniform(-0.1,0.1,(units,129))
        for i in range(4):
            alpha[i][0] = 0
        beta = np.random.uniform(-0.1,0.1,(10,units+1))
        for i in range(10):
            beta[i][0] = 0
    if(a == 2):
        #zero, hidden layer(D) = 4, length of vector M+1 = 129
        alpha = np.zeros((4,129))
        # small dataset, K = 10, D = 4
        beta = np.zeros((10,5))
        
    learningRate = lr
    return alpha, beta, learningRate

In [1181]:
# read file
def readData(file):
    with open(file, 'rt') as csvfile:
        reader = csv.reader(csvfile)
        rawdata = [row for row in reader]

    output = []
    for i in range(len(rawdata)):
        output.append(int(rawdata[i][0]))

    data = [[] for i in range(len(rawdata))] 
    for i in range(len(rawdata)):
        data[i].append(1)
        for j in range(1,len(rawdata[0])):
            data[i].append(int(rawdata[i][j]))

    input = np.array(data)

    return input, output
  

In [1182]:
trainIn, trainOut = readData("smallTrain.csv")
testIn, testOut = readData("smallTest.csv")

In [1183]:
# Forward
def linearFW1(x, alpha):
    a = np.matmul(alpha, x.T)
    return a

In [1184]:
def sigmoidFW(a):
    z = []
    for i in range(len(a)):
        z.append(1/(1 + np.exp(-a[i])))
    z.insert(0,1)
    z = np.array(z)
    return z

In [1185]:
def linearFW2(z, beta):
    b = np.matmul(beta, z.T)
    return b

In [1186]:
def softmaxFW(b):
    y1 = []
    e = np.exp(b)
    for i in range(len(e)):
        y1.append(e[i]/np.sum(e))
    y1 = np.array(y1)
    return y1

In [1187]:
# Backward
def softmaxBW(y,y1):
    gb = []
    for i in range(len(y)):
        gb.append(y1[i]-y[i])
    gb = np.array(gb)
    return gb

In [1188]:
def linearBW1(z,gb,beta):
    beta1 = np.delete(beta,0,1)
    gbeta = np.matmul(gb, z.reshape(1,len(z)))
    gz = np.matmul(beta1.T, gb)
    return gbeta, gz

gbeta, gz = linearBW1(z,gb,beta)

In [1189]:
def sigmoidBW(z,gz):
    z = z[1:]
    z = z.reshape((len(z),1))
    ga = gz * z *(1 - z)
    return ga

In [1190]:
def linearBW2(x,ga):
    galpha = np.matmul(ga, x.reshape((1,len(x))))
    return galpha

In [1191]:
def update(alpha, beta, galpha, gbeta, lr):
    alpha = alpha - lr*galpha
    beta = beta - lr*gbeta
    return alpha, beta

In [1192]:
# m = [1,1,1,0,0,1,1]
# x = np.array(m)
# n = [[1,1,2,-3,0,1,-3],[1,3,1,2,1,0,2],[1,2,2,2,2,2,1],[1,1,0,2,1,-2,2]]
# alpha = np.array(n)
# h = [[1,1,2,-2,1],[1,1,-1,1,2],[1,3,1,-1,1]]
# beta = np.array(h)
# beta1 = np.delete(beta,0,1)
# k = [0,1,0]
# y = np.array(k)

In [1195]:
def epoch(a, alpha, beta, learningRate):
    
    # number of epoch is a.
    epoch_count = 0
    for k in range(a):
        epoch_count += 1
        for i in range(len(trainIn)):
            x = trainIn[i]
            y = np.zeros((10,1))
            y[trainOut[i]] = 1
            a = linearFW1(x,alpha)
            z = sigmoidFW(a)
            b = linearFW2(z,beta)
            y1 = softmaxFW(b)
            gb = softmaxBW(y,y1)
            gbeta, gz = linearBW1(z,gb,beta)
            ga = sigmoidBW(z, gz)
            galpha = linearBW2(x,ga)
            alpha, beta = update(alpha, beta, galpha, gbeta, learningRate)
            
        # train entropy   
        traincrossEntropy = 0
        for i in range(len(trainIn)):
            x = trainIn[i]
            y = np.zeros((10,1))
            y[trainOut[i]] = 1
            a = linearFW1(x,alpha)
            z = sigmoidFW(a)
            b = linearFW2(z,beta)
            y1 = softmaxFW(b)
            for j in range(10):
                traincrossEntropy += y[j] * np.log(y1[j])
                
        traincrossEntropy = -1/len(trainIn)*traincrossEntropy
        metrics_file = open(metrics,'w')
        metrics_file.write("epoch=" + str(epoch_count) + " crossentropy(train): " + str(traincrossEntropy))
        print("train:")
        print(traincrossEntropy) 
        
        # test entropy
        testcrossEntropy = 0
        for i in range(len(testIn)):
            x = testIn[i]
            y = np.zeros((10,1))
            y[testOut[i]] = 1
            a = linearFW1(x,alpha)
            z = sigmoidFW(a)
            b = linearFW2(z,beta)
            y1 = softmaxFW(b)
            for j in range(10):
                testcrossEntropy += y[j] * np.log(y1[j])
                
        testcrossEntropy = -1/len(testIn)*testcrossEntropy
        metrics_file = open(metrics,'w')
        metrics_file.write("epoch=" + str(epoch_count) + " crossentropy(test): " + str(testcrossEntropy))
        print("test:")
        print(testcrossEntropy)
        
        
    #train error 
    index = []
    trainError = 0
    for i in range(len(trainIn)):
        x = trainIn[i]
        y = np.zeros((10,1))
        y[trainOut[i]] = 1
        a = linearFW1(x,alpha)
        z = sigmoidFW(a)
        b = linearFW2(z,beta)
        y1 = softmaxFW(b)
        m = np.argmax(y1, axis=0)
        index.append(m)
        if (m != trainOut[i]):
            trainError += 1
    trainErrorRate = trainError/len(trainOut)
    metrics_file = open(metrics,'w')
    metrics_file.write('error(train): ' + str(trainErrorRate)+'\n')
    print(trainErrorRate)
    
    #test error 
    testindex = []
    testError = 0
    for i in range(len(testIn)):
        x = testIn[i]
        y = np.zeros((10,1))
        y[testOut[i]] = 1
        a = linearFW1(x,alpha)
        z = sigmoidFW(a)
        b = linearFW2(z,beta)
        y1 = softmaxFW(b)
        n = np.argmax(y1, axis=0)
        testindex.append(n)
        if (n != testOut[i]):
            testError += 1
    testErrorRate = testError/len(testOut)
    metrics_file = open(metrics,'w')
    metrics_file.write('error(test): ' + str(testErrorRate)+'\n')
    print(testErrorRate)
    
    return index, testindex

In [1196]:
alpha_, beta_ , lr_ = model(model_num,hiddenUnits,lr)
k,l = epoch(epoch_num, alpha_, beta_, lr_)

train:
[2.18506276]
test:
[2.18827303]
train:
[1.90103258]
test:
[1.91363803]
0.77
0.78


In [ ]:
trainOut_file = open(train_out,'w')
for s in k:
    trainOut_file.write(s+'\n')

testOut_file = open(test_out,'w')
for t in l:
    testOut_file.write(t+'\n')